In [ ]:
# todo NOTE: this script does not reproduce original result since we removed device IDs 
# todo NOTE: also this script will encounter runtime/space error since we used device IDs as a key while merge dataframes 

import pandas as pd
import numpy as np

file_cleaned_flow = '../../Endpoint Mapping Data/Cleaned Flow/cleaned_flow_stat.csv'
cleaned_flow = pd.read_csv(file_cleaned_flow)

cleaned_flow['super_vendor'] = cleaned_flow.apply(
    lambda row: row.vendor_name.lower() if row.vendor_name==row.vendor_name
    else row.gpt_clean_vendor,
    axis=1)


cleaned_flow['generic_category'] = cleaned_flow.apply(
    lambda row: row.man_generic_category if row.man_generic_category==row.man_generic_category
    else row.gpt_generic_category,
    axis=1)


# read all party mapping files
file_all_party_mapping = '../../Endpoint Mapping Data/Domain Data/all_party_mapping.csv'
all_party_mapping = pd.read_csv(file_all_party_mapping)
# drop extra columns 
all_party_mapping = all_party_mapping[['super_vendor', 'domain', 'party_labels']].drop_duplicates()

# marge with clean flow
clean_flow_party_label = pd.merge(cleaned_flow,
                                  all_party_mapping,
                                  on=['super_vendor', 'domain'],
                                  how='left'
                                  )


# read user information file 
file_user_device_timezone = '../../Inspector Dataset/New data/user_device_timezone.csv'
user_device_timezone = pd.read_csv(file_user_device_timezone)
user_device_timezone = user_device_timezone[['device_id', 'user_key', 'user_country', 'timezone']]

# merge with timezone file 
clean_flow_party_label = pd.merge(clean_flow_party_label,
                                  user_device_timezone,
                                  on=['device_id'],
                                  how='left'
                                  )


unique_categories = ['Media/TV',
                     'Voice Assistant',
                     'Surveillance',
                     'Home Automation',
                     'Home Appliance ',
                     'Game Console',
                     'Generic IoT',
                     'Work Appliance',
                     'Vehicle']

devices_df = clean_flow_party_label

## RQ 2
##### How frequently does user data cross borders?


In [ ]:
import json
file_location = '../../Statistical Data/RQ2/country-list.json'

with open(file_location, encoding='utf-8') as f:
    country_2_continent = json.load(f)

# Create a list of tuples containing key-value pairs
country_2_continent = [(key, value) for key, value in country_2_continent.items()]
country_2_continent = pd.DataFrame(country_2_continent, columns=['remote_ip_country', 'domain_loc'])

# find continent to remote ip country 
clean_flow_party_label  = pd.merge(clean_flow_party_label, country_2_continent, on=['remote_ip_country'], how='left')


# function to find the timezone of the users
def timezone_continent(timezone):
    try:
        timezone = float(timezone)
        if (timezone >= -11) and (timezone < -1):
            return 'AMERICAS'
        if (timezone >= 0) and (timezone <= 3):
            return 'EA'
        if timezone >= 4:
            return 'APA'
    except:
        pass
    return 'UN'


timezone = clean_flow_party_label[['device_id', 'timezone']].drop_duplicates()
timezone['user_loc'] =  timezone.apply(lambda row: timezone_continent(row.timezone), axis=1)

# find user location based on timezone

clean_flow_party_label = pd.merge(clean_flow_party_label, timezone, on=['device_id', 'timezone'], how='left')
location = ['AMERICAS', 'EA', 'APA']

def flow_over_boarder_updated(dataframe):
    for user_loc in location:
        user_df = dataframe[dataframe['user_loc']==user_loc]
        total_out_byte = user_df[user_df['domain_loc']=='AMERICAS']['total_out_byte'].sum() + user_df[user_df['domain_loc']=='EA']['total_out_byte'].sum() + user_df[user_df['domain_loc']=='APA']['total_out_byte'].sum()

        print('{', end='')
        for domain_loc in location:
            out_byte = user_df[user_df['domain_loc']==domain_loc]['total_out_byte'].sum()
            flow_percentage = out_byte*100/total_out_byte

            if domain_loc != 'APA': print('%.1f' % flow_percentage, end=', ')
            else: print('%.1f' % flow_percentage, end='},\n')



### Printing results for RQ3 (Figure 5)
for category in unique_categories:
    df = clean_flow_party_label[(clean_flow_party_label['generic_category']==category)]
    print(category)
    flow_over_boarder_updated(df)

In [ ]:
def flow_over_boarder_different_party(dataframe):
    for user_loc in location:
        user_df = dataframe[dataframe['user_loc']==user_loc]
        # total_out_byte = user_df['total_out_byte'].sum()
        total_out_byte = user_df[user_df['domain_loc']=='AMERICAS']['total_out_byte'].sum() + user_df[user_df['domain_loc']=='EA']['total_out_byte'].sum() + user_df[user_df['domain_loc']=='APA']['total_out_byte'].sum()
        
        print('{', end='')
        for domain_loc in location:
            out_byte_1 = user_df[(user_df['domain_loc']==domain_loc) & (user_df['party_labels']==1)]['total_out_byte'].sum()
            flow_percentage_1 = out_byte_1*100/total_out_byte
            
            out_byte_2 = user_df[(user_df['domain_loc']==domain_loc) & (user_df['party_labels']==2)]['total_out_byte'].sum()
            flow_percentage_2 = out_byte_2*100/total_out_byte
            
            out_byte_3 = user_df[(user_df['domain_loc']==domain_loc) & (user_df['party_labels']==3)]['total_out_byte'].sum()
            flow_percentage_3 = out_byte_3*100/total_out_byte
            
            if domain_loc != 'APA': print('%.1f' % flow_percentage_1, ', ', '%.1f' % flow_percentage_2, ', ', '%.1f' % flow_percentage_3, end=', ')
            else: print('%.1f' % flow_percentage_1, ', ', '%.1f' % flow_percentage_2, ', ', '%.1f' % flow_percentage_3, end='},\n')
            
### Printing results for RQ3 (Figure 8)
for category in unique_categories:
    df = clean_flow_party_label[clean_flow_party_label['generic_category']==category]
    print(category)
    flow_over_boarder_different_party(df)